<a href="https://colab.research.google.com/github/golshidr/Augmentation-methods-datasets/blob/main/bankruptcy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **import library**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore', '.*do not.*', )
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: DelftStack
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
pip install impyute

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install fancyimpute


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 40.9 MB/s 
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29899 sha256=ece335a7f21c721d504917e3f0b2b0045a8d556e17eb590a9ff4e5dbcdc27e08
  Stored in directory: /root/.cache/pip/wheels/e3/04/06/a1a7d89ef4e631ce6268ea2d8cde04f7290651c1ff1025ce68
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11353 sha256=7d520e3f0015f771b105752a9096022720f3cef1d4079fb653084786d20301b0
  Stored in directory: /root/.cache/pip/wheels/72/21/a8/a045cacd9838abd5643f6bfa852c0796a99d6b1494760494e0
Successfully built fancyimpute knnimpute


In [ ]:
pip install arff

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for arff: filename=arff-0.9-py3-none-any.whl size=4971 sha256=ee170cadc2e552c0ab0a3e17e39ca33fa6b815ed7521694648b0246c446e4399
  Stored in directory: /root/.cache/pip/wheels/fe/81/bd/4ae90e24ba860304e375da219f9205b2586dbee255f3ee70e2
Successfully built arff


In [ ]:
from sklearn.impute import SimpleImputer
#imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
import pandas as pd
import numpy as np
#To perform kFold Cross Validation
from sklearn.model_selection import KFold
# Formatted counter of class labels
from collections import Counter
# Ordered Dictionary
from collections import OrderedDict
# Library imbalanced-learn to deal with the data imbalance. To use SMOTE oversampling
from imblearn.over_sampling import SMOTE 

# Impoting classification models
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

import random

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

In [ ]:
from scipy.sparse import data
from scipy.io import arff
############################################################
# Loads the 5 raw .arff files into a list
def load_arff_raw_data():
    N=5
    return [arff.loadarff('drive/MyDrive/' + str(i+1) + 'year.arff') for i in range(N)]

############################################################
# Loads the 5 raw .arff files into pandas dataframes
def load_dataframes():
    return [pd.DataFrame(data_i_year[0]) for data_i_year in load_arff_raw_data()]

############################################################
# Set the column headers from X1 ... X64 and the class label as Y, for all the 5 dataframes.
def set_new_headers(dataframes):
    cols = ['X' + str(i+1) for i in range(len(dataframes[0].columns)-1)]
    cols.append('Y')
    for df in dataframes:
        df.columns = cols

############################################################
# dataframes is the list of pandas dataframes for the 5 year datafiles.  
dataframes = load_dataframes()

# Set the new headers for the dataframes. The new headers will have the renamed set of feature (X1 to X64)
set_new_headers(dataframes)    

# print the first 5 rows of a dataset 'year1'
dataframes[0].describe()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64
count,7024.000000,7024.000000,7024.000000,6997.000000,7.019000e+03,7024.000000,7024.000000,7002.000000,7026.000000,7024.000000,...,7.027000e+03,7.027000e+03,7026.000000,7.027000e+03,7026.000000,6.892000e+03,7005.000000,7.027000e+03,6997.000000,6993.000000
mean,0.034660,0.560215,0.119969,2.629143,-2.631672e+02,0.059712,0.313876,2.623996,5.552855,1.825832,...,8.855693e+03,-1.577367e+02,0.193243,1.587409e+02,0.277829,4.328830e+02,15.642228,4.763202e+03,8.126852,208.731950
std,4.565504,5.350084,5.275459,13.257356,3.707460e+04,6.051113,8.353274,18.708327,101.995448,33.836452,...,7.247527e+04,1.322125e+04,4.344046,1.322124e+04,6.339149,2.612802e+04,261.554534,3.107835e+05,19.996419,5140.708804
min,-256.890000,-72.162000,-440.500000,0.000000,-2.722100e+06,-397.890000,-189.560000,-141.410000,0.000005,-440.550000,...,-8.004700e+05,-1.108300e+06,-315.370000,-4.194000e-03,-327.970000,4.700000e-05,0.000016,0.000000e+00,0.000015,0.000010
25%,0.021182,0.296678,0.026968,1.063100,-4.449800e+01,0.000000,0.028023,0.445710,1.037225,0.300785,...,9.712000e+01,2.031450e-02,0.056772,8.647650e-01,0.000000,5.923950e+00,4.829000,4.322250e+01,3.425400,2.538600
50%,0.075802,0.482960,0.181275,1.502000,-5.373900e+00,0.000000,0.090109,1.015100,1.205750,0.492235,...,1.604800e+03,6.338200e-02,0.175745,9.388100e-01,0.028438,1.004050e+01,7.033700,6.850900e+01,5.303200,4.637700
75%,0.160268,0.680233,0.362548,2.460700,3.777050e+01,0.146660,0.188667,2.267675,2.132975,0.675677,...,5.955900e+03,1.376950e-01,0.351922,9.820150e-01,0.273867,2.013900e+01,10.703000,1.063350e+02,8.357900,9.782200
max,94.280000,441.500000,1.000000,1017.800000,9.909000e+05,303.670000,453.770000,1452.200000,3876.100000,1099.500000,...,4.398400e+06,1.000000e+00,126.670000,1.108300e+06,119.580000,2.137800e+06,21110.000000,2.501600e+07,1042.200000,294770.000000


In [ ]:
# Convert the dtypes of all the columns (other than the class label columns) to float.
def convert_columns_type_float(dfs):
    for i in range(5):
        index = 1
        while(index<=63):
            colname = dfs[i].columns[index]
            col = getattr(dfs[i], colname)
            dfs[i][colname] = col.astype(float)
            index+=1
            
convert_columns_type_float(dataframes) 

In [ ]:
# The class labels for all the dataframes are originally in object type.
# Convert them to int types
def convert_class_label_type_int(dfs):
    for i in range(len(dfs)):
        col = getattr(dfs[i], 'Y')
        dfs[i]['Y'] = col.astype(int)
        
convert_class_label_type_int(dataframes)
dataframes[0].shape

(7027, 65)

In [ ]:
#missing data using mean strategy
def perform_mean_imputation(dfs):
    # Construct an imputer with strategy as 'mean', to mean-impute along the columns
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    mean_imputed_dfs = [pd.DataFrame(imputer.fit_transform(df)) for df in dfs]
    for i in range(len(dfs)):
        mean_imputed_dfs[i].columns = dfs[i].columns   
    return mean_imputed_dfs

mean_imputed_dataframes = perform_mean_imputation(dataframes)
dataframes[0].shape


(7027, 65)

In [ ]:
# dealing with nan data using mean strategy
def claculate_nan(dfs):
    for i in range(len(dfs)):
      index=0
      while(index<=63):
        m = dfs[i].mean(axis=0)
        dfs[i] = dfs[i].fillna(m)
        index=index+1

claculate_nan = claculate_nan(dataframes)
dataframes[0].describe()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X56,X57,X58,X59,X60,X61,X62,X63,X64,Y
count,7027.000000,7027.000000,7027.000000,7027.000000,7.027000e+03,7027.000000,7027.000000,7027.000000,7027.000000,7027.000000,...,7.027000e+03,7027.000000,7.027000e+03,7027.000000,7.027000e+03,7027.000000,7.027000e+03,7027.000000,7027.000000,7027.000000
mean,0.034660,0.560215,0.119969,2.629143,-2.631672e+02,0.059712,0.313876,2.623996,5.552855,1.825832,...,-1.577367e+02,0.193243,1.587409e+02,0.277829,4.328830e+02,15.642228,4.763202e+03,8.126852,208.731950,0.038566
std,4.564529,5.348941,5.274332,13.229022,3.705348e+04,6.049821,8.351490,18.675013,101.988190,33.829227,...,1.322125e+04,4.343737,1.322124e+04,6.338698,2.587579e+04,261.144719,3.107835e+05,19.953682,5128.255341,0.192571
min,-256.890000,-72.162000,-440.500000,0.000000,-2.722100e+06,-397.890000,-189.560000,-141.410000,0.000005,-440.550000,...,-1.108300e+06,-315.370000,-4.194000e-03,-327.970000,4.700000e-05,0.000016,0.000000e+00,0.000015,0.000010,0.000000
25%,0.021208,0.296760,0.027011,1.064100,-4.462850e+01,0.000000,0.028037,0.447340,1.037250,0.300900,...,2.031450e-02,0.056814,8.647650e-01,0.000000,6.013050e+00,4.834250,4.322250e+01,3.432500,2.550300,0.000000
50%,0.075790,0.483090,0.181180,1.505200,-5.575300e+00,0.000000,0.090177,1.022100,1.206000,0.492450,...,6.338200e-02,0.175820,9.388100e-01,0.028451,1.024900e+01,7.046300,6.850900e+01,5.327800,4.657400,0.000000
75%,0.160240,0.680170,0.362500,2.476050,3.769500e+01,0.146570,0.188880,2.286700,2.133150,0.675915,...,1.376950e-01,0.351895,9.820150e-01,0.274140,2.136650e+01,10.775500,1.063350e+02,8.326800,9.912750,0.000000
max,94.280000,441.500000,1.000000,1017.800000,9.909000e+05,303.670000,453.770000,1452.200000,3876.100000,1099.500000,...,1.000000e+00,126.670000,1.108300e+06,119.580000,2.137800e+06,21110.000000,2.501600e+07,1042.200000,294770.000000,1.000000


In [ ]:
df=dataframes[0].append(dataframes[1])

In [ ]:
df1=df.append(dataframes[2])

In [ ]:
df2=df1.append(dataframes[3])

In [ ]:
df3=df2.append(dataframes[4])

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/My Drive/pankruptcy_poland_dataset.csv')
dd=df[df['Y'] == 1]
dd.shape

(2091, 66)

In [ ]:
df

(7027, 65)

# Naive approach

In [ ]:
def naive_approach(dfs):
    for i in range(0):
        index = 0
        df_1 = dfs[i].loc[dfs[i]['Y'] == 1]
        df_0 = dfs[i].loc[dfs[i]['Y'] == 0]
        n=(len(df_0)-len(df_1))
        new_data=pd.DataFrame()
        dfs_0=pd.DataFrame()
        print('Shape of dataframe[0] in the first step:',dataframes[0].shape)
        j=0
        while(index<=64):
            colname = df_1.columns[index]
            col = getattr(df_1, colname)
            max_value = col.max()
            min_value = col.min()
            random_float_list = []
            j=0
            # Generating data in each column
            for i in range(0, n):
              random_float_list.append(round(random.uniform(min_value, max_value), 5))
              
            np.reshape(random_float_list,(n,1))
            new_data[df_1.columns[index]]=random_float_list
            
            index+=1
            
    new_data[new_data.columns[-1]] == 1
    df_1=df_1.append(new_data)
    dfs_0=df_0.append(df_1)
    
    print('shape of new data that has been generated:',new_data.shape)
    print('Shape of class with label 1: ',df_1.shape)
    print('Shape of class with label 0: ',df_0.shape)
    print('Shape of dfs_0 at last: ',dfs_0.shape)
    path = '/content/drive/My Drive/year1.csv'
    with open(path, 'w', encoding = 'utf-8-sig') as f:
      dfs_0.to_csv(f)

naive_approach=naive_approach(dataframes) 

UnboundLocalError: ignored

In [ ]:
def mean_approach(dfs):
    for i in range(1):
        index = 0
        df_1 = dfs[i].loc[dfs[i]['Y'] == 1]
        df_0 = dfs[i].loc[dfs[i]['Y'] == 0]
        n=(len(df_0)-len(df_1))
        new_data=pd.DataFrame()
        dfs_0=pd.DataFrame()
        print('Shape of dataframe[0] in the first step:',dataframes[0].shape)
        j=0
        while(index<=64):
            colname = df_1.columns[index]
            col = getattr(df_1, colname)
            mean_value = col.mean()
            max_value = col.max()
            std_value=col.std()
            min_value = col.min()
            mean_float_list = []
            j=0
            # Generating data in each column
            for i in range(0, n):
              z= mean_value - std_value
              mean_float_list.append(z)
              

            random.shuffle(mean_float_list)  
            np.reshape(mean_float_list,(n,1))
            new_data[df_1.columns[index]]=mean_float_list
            
            index+=1
            
    new_data[new_data.columns[-1]] == 1
    df_1=df_1.append(new_data)
    dfs_0=df_0.append(df_1)
    
    #print('shape of new data that has been generated:',new_data.shape)
    #print('Shape of class with label 1: ',df_1.shape)
    #print('Shape of class with label 0: ',df_0.shape)
    #print('Shape of dfs_0 at last: ',dfs_0.shape)
    path = '/content/drive/My Drive/year1.csv'
    with open(path, 'w', encoding = 'utf-8-sig') as f:
      dfs_0.to_csv(f)
    print(dfs_0.describe())

mean_approach=mean_approach(dataframes) 

Shape of dataframe[0] in the first step: (7027, 65)
                 X1            X2            X3            X4            X5  \
count  13512.000000  13512.000000  13512.000000  13512.000000  1.351200e+04   
mean      -1.880888    -11.458265    -13.547695     -5.406901 -5.198312e+03   
std        3.848492     13.092215     14.727490     12.688094  2.720958e+04   
min     -256.890000    -72.162000   -440.500000    -14.114577 -2.722100e+06   
25%       -3.956533    -24.481219    -28.357668    -14.114577 -1.054592e+04   
50%       -0.086261      0.079781     -0.265160      0.557340 -3.514500e+02   
75%        0.080474      0.498965      0.195227      1.550525 -2.588225e+00   
max       94.280000    441.500000      1.000000   1017.800000  9.909000e+05   

                 X6            X7            X8            X9           X10  \
count  13512.000000  13512.000000  13512.000000  13512.000000  13512.000000   
mean     -12.298663     -1.729587     -5.547964      2.310669    -12.544845   

# SMOTE

In [ ]:
#SMOTE

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix

class_0_org = dataframes[4][dataframes[4]['Y'] == 0]
class_1_org = dataframes[4][dataframes[4]['Y'] == 1]


df=dataframes[4]
print('the shape of data befor oversampling:',df.shape)

X=df.iloc[:,1:64].values
Y=df.iloc[:,-1].values

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X, Y = sm.fit_resample(X, Y.ravel())

X=pd.DataFrame(X)
Y=pd.DataFrame(Y)
df = pd.concat([X, Y], axis=1)
print('the shape of data after oversampling:',df.shape)
df.to_csv('/content/drive/My Drive/year4.csv')
class_0 = df[df.iloc[:,-1] == 0]
class_1 = df[df.iloc[:,-1] == 1]




for j in range(0,len(class_1_org),100):
  class_1_test=class_1_org.iloc[j:j+50] # extract from orginal data
  class_1 = class_1.loc[:,~class_1.columns.duplicated()].reset_index(drop=True)
  class_1_train=pd.concat([class_1, class_1_test]).drop_duplicates(keep=False)
  class_1_train.drop(class_1_train.iloc[:, 63:], inplace = True, axis = 1)
  class_1_train["Y"] = 1

  

  class_0_test=class_0_org.iloc[j:j+50] # extract from orginal data
  class_0 = class_0.loc[:,~class_0.columns.duplicated()].reset_index(drop=True)
  class_0_train=pd.concat([class_0, class_0_test]).drop_duplicates(keep=False)
  class_0_train.drop(class_0_train.iloc[:, 63:], inplace = True, axis = 1)
  class_0_train["Y"] = 0


  train_data = class_1_train.append(class_0_train)
  test_data = class_1_test.append(class_0_test)


  
  X_train=train_data.iloc[:,1:63].values
  X_train=pd.DataFrame(X_train)
  X_train = X_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)



  Y_train=train_data.iloc[:,-1].values
  Y_train=pd.DataFrame(Y_train)
  Y_train = Y_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)




  X_test=test_data.iloc[:,1:63].values
  
  X_test=pd.DataFrame(X_test)
  
  Y_test=test_data.iloc[:,-1].values
  Y_test=pd.DataFrame(Y_test)


  model = KNeighborsClassifier(n_neighbors=5)
  model.fit(X_train,Y_train.values.ravel())
  Y_pred_train=model.predict(X_train)
  Y_pred_test=model.predict(X_test)



  print('Test: consfusion matrix :\n',confusion_matrix(Y_test,Y_pred_test))



the shape of data befor oversampling: (5910, 65)
the shape of data after oversampling: (11000, 64)
Test: consfusion matrix :
 [[29 21]
 [12 38]]
Test: consfusion matrix :
 [[ 9 41]
 [11 39]]
Test: consfusion matrix :
 [[36 14]
 [15 35]]
Test: consfusion matrix :
 [[10 40]
 [11 39]]
Test: consfusion matrix :
 [[43  7]
 [ 5  5]]


# K-Means-SMOTE

In [ ]:
"""K-Means SMOTE oversampling method for class-imbalanced data"""

# Authors: Felix Last
# License: MIT

import warnings
import math
import copy
import numpy as np

from sklearn.utils import check_random_state
from sklearn.metrics.pairwise import euclidean_distances

from imblearn.over_sampling.base import BaseOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.exceptions import raise_isinstance_error
from imblearn.utils import check_neighbors_object
from imblearn.utils.deprecation import deprecate_parameter

class KMeansSMOTE(BaseOverSampler):
    """Class to perform oversampling using K-Means SMOTE.

    K-Means SMOTE works in three steps:

    1. Cluster the entire input space using k-means.
    2. Distribute the number of samples to generate across clusters:

        1. Select clusters which have a high number of minority class samples.
        2. Assign more synthetic samples to clusters where minority class samples are sparsely distributed.

    3. Oversample each filtered cluster using SMOTE.

    The method implements SMOTE and random oversampling as limit cases. Therefore, the following configurations
    may be used to achieve the behavior of ...

    ... SMOTE: ``imbalance_ratio_threshold=float('Inf'), kmeans_args={'n_clusters':1}``

    ... random oversampling: ``imbalance_ratio_threshold=float('Inf'), kmeans_args={'n_clusters':1}, smote_args={'k_neighbors':0})``

    Parameters
    ----------
    sampling_strategy : str, dict, or callable, optional (default='auto')
        Ratio to use for resampling the data set.

        - If ``str``, has to be one of: (i) ``'minority'``: resample the
          minority class; (ii) ``'majority'``: resample the majority class,
          (iii) ``'not minority'``: resample all classes apart of the minority
          class, (iv) ``'all'``: resample all classes, and (v) ``'auto'``:
          correspond to ``'all'`` with for oversampling methods and ``'not
          minority'`` for undersampling methods. The classes targeted will be
          oversampled or undersampled to achieve an equal number of sample
          with the majority or minority class.
        - If ``dict``, the keys correspond to the targeted classes. The values
          correspond to the desired number of samples.
        - If callable, function taking ``y`` and returns a ``dict``. The keys
          correspond to the targeted classes. The values correspond to the
          desired number of samples.

    random_state : int, RandomState instance or None, optional (default=None)
        If int, ``random_state`` is the seed used by the random number
        generator; If ``RandomState`` instance, random_state is the random
        number generator; If ``None``, the random number generator is the
        ``RandomState`` instance used by ``np.random``.
        Will be copied to kmeans_args and smote_args if not explicitly passed there.

    kmeans_args : dict, optional (default={})
        Parameters to be passed to ``sklearn.cluster.KMeans`` or ``sklearn.cluster.MiniBatchKMeans``
        (see ``use_minibatch_kmeans``). If n_clusters is not explicitly set, scikit-learn's
        default will apply.

    smote_args : dict, optional (default={})
        Parameters to be passed to ``imblearn.over_sampling.SMOTE``. Note that ``k_neighbors`` is automatically
        adapted without warning when a cluster is smaller than the number of neighbors specified.
        `sampling_strategy` will be overwritten according to sampling_strategy passed to this class. `random_state`
        will be passed from this class if none is specified.

    imbalance_ratio_threshold : float or dict, optional (default=1.0)
        Specify a threshold for a cluster's imbalance ratio  ``((majority_count + 1) / (minority_count + 1))``.
        Only clusters with an imbalance ratio less than the threshold are oversampled. Use a dictionary to specify
        different thresholds for different minority classes.

    density_power : float, optional (default=None)
        Used to compute the density of minority samples within each cluster. By default, the number of features will be used.

    use_minibatch_kmeans : boolean, optional (default=True)
        If False, use ``sklearn.cluster.KMeans``. If True, use ``sklearn.cluster.MiniBatchKMeans``.

    n_jobs : int, optional (default=1)
        The number of threads to open if possible. This parameter will be copied to ``kmeans_args`` and
        ``smote_args`` if not explicitly passed there. Note: ``MiniBatchKMeans`` does not accept ``n_jobs``.

    Examples
    --------


    """

    def __init__(self,
                sampling_strategy='auto',
                random_state=None,
                kmeans_args=None,
                smote_args=None,
                imbalance_ratio_threshold=1.0,
                density_power=None,
                use_minibatch_kmeans=True,
                n_jobs=1,
                **kwargs):
        super(KMeansSMOTE, self).__init__(sampling_strategy=sampling_strategy, **kwargs)
        if kmeans_args is None:
            kmeans_args = {}
        if smote_args is None:
            smote_args = {}
        self.imbalance_ratio_threshold = imbalance_ratio_threshold
        self.kmeans_args = copy.deepcopy(kmeans_args)
        self.smote_args = copy.deepcopy(smote_args)
        self.random_state = random_state
        self.n_jobs = n_jobs
        self.use_minibatch_kmeans = use_minibatch_kmeans

        self.density_power = density_power

    def _cluster(self, X):
        """Run k-means to cluster the dataset

        Parameters
        ----------
        X : ndarray, shape (n_samples, n_features)
            Matrix containing the data which have to be sampled.

        Returns
        -------
        cluster_assignment : ndarray, shape (n_samples)
            The corresponding cluster labels of ``X``.
        """

        if self.use_minibatch_kmeans:
            from sklearn.cluster import MiniBatchKMeans as KMeans
        else:
            from sklearn.cluster import KMeans as KMeans

        kmeans = KMeans(**self.kmeans_args)
        if self.use_minibatch_kmeans and 'init_size' not in self.kmeans_args:
            self.kmeans_args['init_size'] = min(2 * kmeans.n_clusters, X.shape[0])
            kmeans = KMeans(**self.kmeans_args)

        kmeans.fit_transform(X)
        cluster_assignment = kmeans.labels_
        # kmeans.labels_ does not use continuous labels,
        # i.e. some labels in 0..n_clusters may not exist. Tidy up this mess.
        return cluster_assignment

    def _filter_clusters(self, X, y, cluster_assignment, minority_class_label):
        """Determine sampling weight for each cluster.

        Parameters
        ----------
        X : ndarray, shape (n_samples, n_features)
            Matrix containing the data which have to be sampled.
        y : ndarray, shape (n_samples, )
            Corresponding label for each sample in X.
        cluster_assignment : ndarray, shape (n_samples)
            The corresponding cluster labels of ``X``.
        minority_class_label : int
            Label of the minority class to filter by.

        Returns
        -------
        sampling_weights : ndarray, shape (np.max(np.unique(cluster_assignment)),)
            Vector of sampling weights for each cluster
        """
        # compute the shape of the density factors
        # since the cluster labels are not continuous, make it large enough
        # to fit all values up to the largest cluster label
        largest_cluster_label = np.max(np.unique(cluster_assignment))
        sparsity_factors = np.zeros((largest_cluster_label + 1,), dtype=np.float64)
        minority_mask = (y == minority_class_label)
        sparsity_sum = 0
        imbalance_ratio_threshold = self.imbalance_ratio_threshold
        if isinstance(imbalance_ratio_threshold, dict):
            imbalance_ratio_threshold = imbalance_ratio_threshold[minority_class_label]

        for i in np.unique(cluster_assignment):
            cluster = X[cluster_assignment == i]
            mask = minority_mask[cluster_assignment == i]
            minority_count = cluster[mask].shape[0]
            majority_count = cluster[~mask].shape[0]
            imbalance_ratio = (majority_count + 1) / (minority_count + 1)
            if (imbalance_ratio < imbalance_ratio_threshold) and (minority_count > 1):
                distances = euclidean_distances(cluster[mask])
                non_diagonal_distances = distances[
                    ~np.eye(distances.shape[0], dtype=np.bool)
                ]
                average_minority_distance = np.mean( non_diagonal_distances )
                if average_minority_distance is 0: average_minority_distance = 1e-1 # to avoid division by 0
                density_factor = minority_count / (average_minority_distance ** self.density_power)
                sparsity_factors[i] = 1 / density_factor

        # prevent division by zero; set zero weights in majority clusters
        sparsity_sum = sparsity_factors.sum()
        if sparsity_sum == 0:
            sparsity_sum = 1 # to avoid division by zero
        sparsity_sum = np.full(sparsity_factors.shape, sparsity_sum, np.asarray(sparsity_sum).dtype)
        sampling_weights = (sparsity_factors / sparsity_sum)

        return sampling_weights


    def _fit_resample(self, X, y):
        """Resample the dataset.

        Parameters
        ----------
        X : ndarray, shape (n_samples, n_features)
            Matrix containing the data which have to be sampled.

        y : ndarray, shape (n_samples, )
            Corresponding label for each sample in X.

        Returns
        -------
        X_resampled : ndarray, shape (n_samples_new, n_features)
            The array containing the resampled data.

        y_resampled : ndarray, shape (n_samples_new)
            The corresponding labels of ``X_resampled``

        """
        self._set_subalgorithm_params()

        if self.density_power is None:
            self.density_power = X.shape[1]

        resampled = [ (X.copy(), y.copy()) ]
        sampling_ratio = {k: v for k, v in self.sampling_strategy_.items()}
        # sampling_strategy_ does not contain classes where n_samples 0
        for class_label in np.unique(y):
            if class_label not in sampling_ratio:
                sampling_ratio[class_label] = 0
        for minority_class_label, n_samples in sampling_ratio.items():
            if n_samples == 0:
                continue

            cluster_assignment = self._cluster(X)
            sampling_weights = self._filter_clusters(X, y, cluster_assignment, minority_class_label)
            smote_args = self.smote_args.copy()
            if np.count_nonzero(sampling_weights) > 0:
                # perform k-means smote
                for i in np.unique(cluster_assignment):
                    cluster_X = X[cluster_assignment == i]
                    cluster_y = y[cluster_assignment == i]
                    if sampling_weights[i] > 0:
                        # determine ratio for oversampling the current cluster
                        target_ratio = {label: np.count_nonzero(cluster_y == label) for label in sampling_ratio}
                        cluster_minority_count = np.count_nonzero(cluster_y == minority_class_label)
                        generate_count = int(round(n_samples * sampling_weights[i]))
                        target_ratio[minority_class_label] = generate_count + cluster_minority_count

                        # make sure that cluster_y has more than 1 class, adding a random point otherwise
                        remove_index = -1
                        if np.unique(cluster_y).size < 2:
                            remove_index = cluster_y.size
                            cluster_X = np.append(cluster_X, np.zeros((1,cluster_X.shape[1])), axis=0)
                            majority_class_label = next( key for key in sampling_ratio.keys() if key != minority_class_label )
                            target_ratio[majority_class_label] = 1 + target_ratio[majority_class_label]
                            cluster_y = np.append(cluster_y, np.asarray(majority_class_label).reshape((1,)), axis=0)

                        # clear target ratio of labels not present in cluster
                        for label in list(target_ratio.keys()):
                            if label not in cluster_y:
                                del target_ratio[label]

                        # modify copy of the user defined smote_args to reflect computed parameters
                        smote_args['sampling_strategy'] = target_ratio

                        smote_args = self._validate_smote_args(smote_args, cluster_minority_count)
                        oversampler = SMOTE(**smote_args)

                        # if k_neighbors is 0, perform random oversampling instead of smote
                        if 'k_neighbors' in smote_args and smote_args['k_neighbors'] == 0:
                                oversampler_args = {}
                                if 'random_state' in smote_args:
                                    oversampler_args['random_state'] = smote_args['random_state']
                                oversampler = RandomOverSampler(**oversampler_args)

                        # finally, apply smote to cluster
                        with warnings.catch_warnings():
                            # ignore warnings about minority class getting bigger than majority class
                            # since this would only be true within this cluster
                            warnings.filterwarnings(action='ignore', category=UserWarning, message=r'After over-sampling, the number of samples \(.*\) in class .* will be larger than the number of samples in the majority class \(class #.* \-\> .*\)')
                            cluster_resampled_X, cluster_resampled_y = oversampler.fit_resample(cluster_X, cluster_y)

                        if remove_index > -1:
                            # since SMOTE's results are ordered the same way as the data passed into it,
                            # the temporarily added point is at the same index position as it was added.
                            for l in [cluster_resampled_X, cluster_resampled_y, cluster_X, cluster_y]:
                                np.delete(l, remove_index, 0)

                        # add new generated samples to resampled
                        resampled.append( (
                            cluster_resampled_X[cluster_y.size:,:],
                            cluster_resampled_y[cluster_y.size:]))
            else:
                # all weights are zero -> perform regular smote
                warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
                target_ratio = {label: np.count_nonzero(y == label) for label in sampling_ratio}
                target_ratio[minority_class_label] = sampling_ratio[minority_class_label]
                minority_count = np.count_nonzero(y == minority_class_label)
                smote_args = self._validate_smote_args(smote_args, minority_count)
                oversampler = SMOTE(**smote_args)
                X_smote, y_smote = oversampler.fit_resample(X, y)
                resampled.append((
                    X_smote[y.size:,:],
                    y_smote[y.size:]))


        resampled = list(zip(*resampled))
        if(len(resampled) > 0):
            X_resampled = np.concatenate(resampled[0], axis=0)
            y_resampled = np.concatenate(resampled[1], axis=0)
        return X_resampled, y_resampled


    def _validate_smote_args(self, smote_args, minority_count):
        # determine max number of nearest neighbors considering sample size
        max_k_neighbors =  minority_count - 1
        # check if max_k_neighbors is violated also considering smote's default
        smote = SMOTE(**smote_args)
        if smote.k_neighbors > max_k_neighbors:
            smote_args['k_neighbors'] = max_k_neighbors
            smote = SMOTE(**smote_args)
        return smote_args

    def _set_subalgorithm_params(self):
        # copy random_state to sub-algorithms
        if self.random_state is not None:
            if 'random_state' not in self.smote_args:
                    self.smote_args['random_state'] = self.random_state
            if 'random_state' not in self.kmeans_args:
                self.kmeans_args['random_state'] = self.random_state

        # copy n_jobs to sub-algorithms
        if self.n_jobs is not None:
            if 'n_jobs' not in self.smote_args:
                    self.smote_args['n_jobs'] = self.n_jobs
            if 'n_jobs' not in self.kmeans_args:
                if not self.use_minibatch_kmeans:
                    self.kmeans_args['n_jobs'] = self.n_jobs

In [ ]:
import numpy as np
from imblearn.datasets import fetch_datasets
#from kmeans_smote import KMeansSMOTE

df=dataframes[0]
print(df.shape)
print(df.head(3))
X=df.iloc[:,1:64].values
y=df.iloc[:,-1].values

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix

class_0_org = dataframes[0][dataframes[0]['Y'] == 0]
class_1_org = dataframes[0][dataframes[0]['Y'] == 1]

[print('Class {} has {} instances'.format(label, count))
 for label, count in zip(*np.unique(y, return_counts=True))]

kmeans_smote = KMeansSMOTE(
    kmeans_args={
        'n_clusters': 100
    },
    smote_args={
        'k_neighbors': 20
    }
)
X_resampled, y_resampled = kmeans_smote.fit_resample(X, y)

[print('Class {} has {} instances after oversampling'.format(label, count))
 for label, count in zip(*np.unique(y_resampled, return_counts=True))]


X=pd.DataFrame(X_resampled)
Y=pd.DataFrame(y_resampled)
df = pd.concat([X, Y], axis=1)
df.to_csv('/content/drive/My Drive/year0.csv')


class_0 = df[df.iloc[:,-1] == 0]
class_1 = df[df.iloc[:,-1] == 1]



for j in range(0,len(class_1_org),500):
  class_1_test=class_1_org.iloc[j:j+50] # extract from orginal data
  class_1 = class_1.loc[:,~class_1.columns.duplicated()].reset_index(drop=True)
  class_1_train=pd.concat([class_1, class_1_test]).drop_duplicates(keep=False)
  class_1_train.drop(class_1_train.iloc[:, 63:], inplace = True, axis = 1)
  class_1_train["Y"] = 1

  

  class_0_test=class_0_org.iloc[j:j+50] # extract from orginal data
  class_0 = class_0.loc[:,~class_0.columns.duplicated()].reset_index(drop=True)
  class_0_train=pd.concat([class_0, class_0_test]).drop_duplicates(keep=False)
  class_0_train.drop(class_0_train.iloc[:, 63:], inplace = True, axis = 1)
  class_0_train["Y"] = 0


  train_data = class_1_train.append(class_0_train)
  test_data = class_1_test.append(class_0_test)


  
  X_train=train_data.iloc[:,1:63].values
  X_train=pd.DataFrame(X_train)
  X_train = X_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)



  Y_train=train_data.iloc[:,-1].values
  Y_train=pd.DataFrame(Y_train)
  Y_train = Y_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)




  X_test=test_data.iloc[:,1:63].values
  
  X_test=pd.DataFrame(X_test)
  
  Y_test=test_data.iloc[:,-1].values
  Y_test=pd.DataFrame(Y_test)


#svclassifier = SVC(kernel='poly')
  #model = KNeighborsClassifier(n_neighbors=5)
  model = BernoulliNB()
  model.fit(X_train,Y_train.values.ravel())
  Y_pred_train=model.predict(X_train)
  Y_pred_test=model.predict(X_test)



  print('Test: consfusion matrix :\n',confusion_matrix(Y_test,Y_pred_test))




(7027, 65)
        X1       X2       X3      X4       X5       X6       X7       X8  \
0  0.20055  0.37951  0.39641  2.0472  32.3510  0.38825  0.24976  1.33050   
1  0.20912  0.49988  0.47225  1.9447  14.7860  0.00000  0.25834  0.99601   
2  0.24866  0.69592  0.26713  1.5548  -1.1523  0.00000  0.30906  0.43695   

       X9      X10  ...      X56      X57      X58       X59     X60     X61  \
0  1.1389  0.50494  ...  0.12196  0.39718  0.87804  0.001924  8.4160  5.1372   
1  1.6996  0.49788  ...  0.12130  0.42002  0.85300  0.000000  4.1486  3.2732   
2  1.3090  0.30408  ...  0.24114  0.81774  0.76599  0.694840  4.9909  3.9510   

       X62     X63      X64  Y  
0   82.658  4.4158   7.4277  0  
1  107.350  3.4000  60.9870  0  
2  134.270  2.7185   5.2078  0  

[3 rows x 65 columns]
Class 0 has 6756 instances
Class 1 has 271 instances


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:305: UserWarning: No minority clusters found for class 1. Performing regular SMOTE. Try changing the number of clusters.


Class 0 has 6756 instances after oversampling
Class 1 has 6756 instances after oversampling
Test: consfusion matrix :
 [[39 11]
 [37 13]]


In [ ]:
from pandas.io.parsers.readers import read_csv
df_aug=read_csv('/content/drive/My Drive/year1.csv')
#df_aug.drop('Unnamed: 0',inplace=True,axis=1)
#df_aug.describe()

# Classifier

In [ ]:
import numpy as np
from imblearn.datasets import fetch_datasets
from pandas.io.parsers.readers import read_csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB

df=dataframes[4]
#df=df.drop('Unnamed: 0',axis=1)

X=df.iloc[:,1:64].values
y=df.iloc[:,-1].values

class_0_org = dataframes[4][dataframes[4]['Y'] == 0]
class_1_org = dataframes[4][dataframes[4]['Y'] == 1]

class_0 = df[df.iloc[:,-1] == 0]
class_1 = df[df.iloc[:,-1] == 1]
f1=0
acc=0
print('SVM in the fifth year:\n')
print('The shape of real data class 0:\n',class_0_org.shape)
print('The shape of real data class 1:\n',class_1_org.shape)
print('The shape of generated data class 0:\n',class_0.shape)
print('The shape of generated data class 1:\n',class_1.shape)
i=0
for j in range(0,len(class_1_org),50):
  i=i+1
  class_1_test=class_1_org.iloc[j:j+50] # extract from orginal data
  class_1 = class_1.loc[:,~class_1.columns.duplicated()].reset_index(drop=True)
  class_1_train=pd.concat([class_1, class_1_test]).drop_duplicates(keep=False)
  class_1_train.drop(class_1_train.iloc[:, 63:], inplace = True, axis = 1)
  class_1_train["Y"] = 1

  

  class_0_test=class_0_org.iloc[j:j+50] # extract from orginal data
  class_0 = class_0.loc[:,~class_0.columns.duplicated()].reset_index(drop=True)
  class_0_train=pd.concat([class_0, class_0_test]).drop_duplicates(keep=False)
  class_0_train.drop(class_0_train.iloc[:, 63:], inplace = True, axis = 1)
  class_0_train["Y"] = 0


  train_data = class_1_train.append(class_0_train)
  test_data = class_1_test.append(class_0_test)


  
  X_train=train_data.iloc[:,1:63].values
  X_train=pd.DataFrame(X_train)
  X_train = X_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)



  Y_train=train_data.iloc[:,-1].values
  Y_train=pd.DataFrame(Y_train)
  Y_train = Y_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)




  X_test=test_data.iloc[:,1:63].values
  
  X_test=pd.DataFrame(X_test)
  
  Y_test=test_data.iloc[:,-1].values
  Y_test=pd.DataFrame(Y_test)


  #model = KNeighborsClassifier(n_neighbors=5)
  #model=RandomForestClassifier(n_estimators=1000)
  #model=SVC(kernel='rbf')
  #model = BernoulliNB()
  #model = GaussianNB()
  #model=MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(35,20), random_state=1,max_iter=1500)
  model=SGDClassifier(loss="hinge", penalty="l2", max_iter=50)


  model.fit(X_train,Y_train.values.ravel())
  Y_pred_train=model.predict(X_train)
  Y_pred_test=model.predict(X_test)



  #print('Test: consfusion matrix :\n',confusion_matrix(Y_test,Y_pred_test))
  f1=f1+f1_score(Y_test,Y_pred_test)
  acc=acc+accuracy_score(Y_test,Y_pred_test)

print(f1/i)
print(acc/i)


SVM in the fifth year:

The shape of real data class 0:
 (5500, 65)
The shape of real data class 1:
 (410, 65)
The shape of generated data class 0:
 (5500, 65)
The shape of generated data class 1:
 (410, 65)
0.31173261681909387
0.4574074074074074


In [ ]:
import numpy as np
from imblearn.datasets import fetch_datasets
from pandas.io.parsers.readers import read_csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing
from sklearn.naive_bayes import BernoulliNB

import numpy as np
from imblearn.datasets import fetch_datasets
#from kmeans_smote import KMeansSMOTE

df=dataframes[4]
print(df.shape)

X=df.iloc[:,1:64].values
y=df.iloc[:,-1].values

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix

class_0_org = df[df['Y'] == 0]
class_1_org = df[df['Y'] == 1]

[print('Class {} has {} instances'.format(label, count))
 for label, count in zip(*np.unique(y, return_counts=True))]

kmeans_smote = KMeansSMOTE(
    kmeans_args={
        'n_clusters': 100
    },
    smote_args={
        'k_neighbors': 20
    }
)
X_resampled, y_resampled = kmeans_smote.fit_resample(X, y)

[print('Class {} has {} instances after oversampling'.format(label, count))
 for label, count in zip(*np.unique(y_resampled, return_counts=True))]


X=pd.DataFrame(X_resampled)
Y=pd.DataFrame(y_resampled)
df = pd.concat([X, Y], axis=1)
df.to_csv('/content/drive/My Drive/year0.csv')


class_0 = df[df.iloc[:,-1] == 0]
class_1 = df[df.iloc[:,-1] == 1]



for j in range(0,len(class_1_org),50):
  class_1_test=class_1_org.iloc[j:j+50] # extract from orginal data
  class_1 = class_1.loc[:,~class_1.columns.duplicated()].reset_index(drop=True)
  class_1_train=pd.concat([class_1, class_1_test]).drop_duplicates(keep=False)
  class_1_train.drop(class_1_train.iloc[:, 63:], inplace = True, axis = 1)
  class_1_train["Y"] = 1

  

  class_0_test=class_0_org.iloc[j:j+50] # extract from orginal data
  class_0 = class_0.loc[:,~class_0.columns.duplicated()].reset_index(drop=True)
  class_0_train=pd.concat([class_0, class_0_test]).drop_duplicates(keep=False)
  class_0_train.drop(class_0_train.iloc[:, 63:], inplace = True, axis = 1)
  class_0_train["Y"] = 0


  train_data = class_1_train.append(class_0_train)
  test_data = class_1_test.append(class_0_test)


  
  X_train=train_data.iloc[:,1:63].values
  X_train=pd.DataFrame(X_train)
  X_train = X_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)



  Y_train=train_data.iloc[:,-1].values
  Y_train=pd.DataFrame(Y_train)
  Y_train = Y_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)




  X_test=test_data.iloc[:,1:63].values
  
  X_test=pd.DataFrame(X_test)
  
  Y_test=test_data.iloc[:,-1].values
  Y_test=pd.DataFrame(Y_test)


#svclassifier = SVC(kernel='poly')
  #model = KNeighborsClassifier(n_neighbors=5)
  model = BernoulliNB()
  model.fit(X_train,Y_train.values.ravel())
  Y_pred_train=model.predict(X_train)
  Y_pred_test=model.predict(X_test)



  print('Test: consfusion matrix :\n',confusion_matrix(Y_test,Y_pred_test))






(5910, 65)
Class 0 has 5500 instances
Class 1 has 410 instances


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:192: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:196: RuntimeWarning: overflow encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:197: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:204: RuntimeWarning: invalid value encountered in true_divide


Class 0 has 5500 instances after oversampling
Class 1 has 410 instances after oversampling
Test: consfusion matrix :
 [[45  5]
 [15 35]]
Test: consfusion matrix :
 [[37 13]
 [13 37]]
Test: consfusion matrix :
 [[38 12]
 [22 28]]
Test: consfusion matrix :
 [[43  7]
 [23 27]]
Test: consfusion matrix :
 [[44  6]
 [16 34]]
Test: consfusion matrix :
 [[43  7]
 [21 29]]
Test: consfusion matrix :
 [[35 15]
 [18 32]]
Test: consfusion matrix :
 [[45  5]
 [14 36]]
Test: consfusion matrix :
 [[37 13]
 [ 3  7]]


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/creditcard.csv')
df_baseline=read_csv('/content/drive/My Drive/creditcard_baseline.csv')
df_baseline=df_baseline.drop('Unnamed: 0',axis=1)

X=df.iloc[:,1:30].values
y=df.iloc[:,-1].values

class_0_org = df[df['Class'] == 0]
class_1_org = df[df['Class'] == 1]

class_0 = df_baseline[df_baseline.iloc[:,-1] == 0]
class_1 = df_baseline[df_baseline.iloc[:,-1] == 1]

print('MLP:\n')
print('The shape of real data class 0:\n',class_0_org.shape)
print('The shape of real data class 1:\n',class_1_org.shape)
print('The shape of generated data class 0:\n',class_0.shape)
print('The shape of generated data class 1:\n',class_1.shape)

for j in range(0,len(class_1_org),50):
  class_1_test=class_1_org.iloc[j:j+50] # extract from orginal data
  class_1 = class_1.loc[:,~class_1.columns.duplicated()].reset_index(drop=True)
  class_1_train=pd.concat([class_1, class_1_test]).drop_duplicates(keep=False)
  class_1_train.drop(class_1_train.iloc[:, 30:], inplace = True, axis = 1)
  class_1_train["Class"] = 1

  

  class_0_test=class_0_org.iloc[j:j+50] # extract from orginal data
  class_0 = class_0.loc[:,~class_0.columns.duplicated()].reset_index(drop=True)
  class_0_train=pd.concat([class_0, class_0_test]).drop_duplicates(keep=False)
  class_0_train.drop(class_0_train.iloc[:, 30:], inplace = True, axis = 1)
  class_0_train["Class"] = 0


  train_data = class_1_train.append(class_0_train)
  test_data = class_1_test.append(class_0_test)


  
  X_train=train_data.iloc[:,1:30].values
  X_train=pd.DataFrame(X_train)
  X_train = X_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)



  Y_train=train_data.iloc[:,-1].values
  Y_train=pd.DataFrame(Y_train)
  Y_train = Y_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)




  X_test=test_data.iloc[:,1:30].values
  
  X_test=pd.DataFrame(X_test)
  
  Y_test=test_data.iloc[:,-1].values
  Y_test=pd.DataFrame(Y_test)


  #model = KNeighborsClassifier(n_neighbors=5)
  model=RandomForestClassifier(n_estimators=1000)
  #model=SVC(kernel='rbf')
  #model = BernoulliNB()
  #model = GaussianNB()
  #model=MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(50,20), random_state=1,max_iter=150)
  #model=SGDClassifier(loss="hinge", penalty="l2", max_iter=50)


  model.fit(X_train,Y_train.values.ravel())
  Y_pred_train=model.predict(X_train)
  Y_pred_test=model.predict(X_test)



  print('Test: consfusion matrix :\n',confusion_matrix(Y_test,Y_pred_test))

MLP:

The shape of real data class 0:
 (284315, 31)
The shape of real data class 1:
 (492, 31)
The shape of generated data class 0:
 (284315, 31)
The shape of generated data class 1:
 (284315, 31)


KeyboardInterrupt: ignored